<a href="https://colab.research.google.com/github/LordFlumpleNerf/BadAssStockPickerAI/blob/main/StockPickerAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### THIS BLOCK IS A CATCH-ALL FOR SHIT TO IMPORT AND SETUP

import matplotlib.pyplot as plt
import yfinance as yf
import alpha_vantage
import requests
import json
#import nasdaq_data_link

import tensorflow as tf
from tensorflow import keras
from keras import models, layers, callbacks
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import Dropout
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
#from sklearn.preprocessing import OneHotEncoder
#from sklearn.model_selection import train_test_split
from datetime import datetime, date, timedelta
import random
import time

import numpy as np
import pandas as pd
import openpyxl
import pytz

%load_ext autoreload
%autoreload 2
%matplotlib inline

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
#pd.set_option('display.width', 300000)
pd.set_option('display.colheader_justify', 'center')
pd.options.mode.chained_assignment = None

In [ ]:
av_api_key = "API_KEY_from_Alpha_Vantage"
alpha_vantage_url = 'https://www.alphavantage.co/query'

sample_size = 320

In [ ]:
#drawing our tickers from iShares ICLN fund or iShares Core Small Cap ETF
### ADDED COLUMN IN EACH UNDER 'usecols' FOR SECTOR

#stocklist = pd.read_excel('drive/My Drive/iShares-Global-Clean-Energy-ETF_fund.xlsx', sheet_name='Holdings', usecols='A,D,E,L', skiprows=7)
#stocklist = pd.read_excel('drive/My Drive/iShares-Core-SP-Small-Cap-ETF_fund.xlsx', sheet_name='Holdings', usecols='A,D,E,L', skiprows=7)
#stocklist = pd.read_excel('drive/My Drive/iShares-Core-SP-Total-US-Stock-Market-ETF_fund.xlsx', sheet_name='Holdings', usecols='A,C,D,K', skiprows=7)

mid_caps = pd.read_excel('drive/My Drive/iShares-Core-SP-Mid-Cap-ETF_fund.xlsx', sheet_name='Holdings', usecols='A,D,E,L', skiprows=7)
large_caps = pd.read_excel('drive/My Drive/iShares-Core-SP-500-ETF_fund.xlsx', sheet_name='Holdings', usecols='A,C,D,K', skiprows=7)

stocklist = pd.concat([large_caps, mid_caps])
stocklist.reset_index(drop=True)

del mid_caps
del large_caps

In [ ]:
#Creating initial ticker list from ICLN or Small-Cap tickers
### CHANGING TO A DATAFRAME TO ADD SECTOR
stocklist = stocklist[(stocklist['Asset Class'] == 'Equity')]
stocklist = stocklist[(stocklist['Exchange'] == 'NASDAQ') | (stocklist['Exchange'] == 'New York Stock Exchange Inc.')]
print(stocklist.head())
print(stocklist.tail())
stocklist = stocklist[['Ticker', 'Sector']].sample(sample_size)

In [ ]:
### one-hot encoding for 'sector'
ticker = stocklist['Ticker']
sector = pd.get_dummies(stocklist['Sector'])
stocklist = pd.concat([ticker, sector], axis=1)
stocklist[sector.columns] = stocklist[sector.columns].astype(float)
stocklist.reset_index(drop=True, inplace=True)

#print(stocklist.head())
del sector


In [ ]:
def get_indices(index_ticker, period):
    # for getting index dataframes for each important index
    ticker = yf.Ticker(index_ticker)

    #initializing the frame
    df = pd.DataFrame(ticker.history(period=period))
    #df.reset_index(drop=True, inplace=True)
    df.drop(columns=['Dividends', 'Stock Splits'], axis=1, inplace=True)
    if index_ticker == '^VIX':
        df.drop(['Volume'], axis=1, inplace=True)

    return df

In [ ]:
def get_cpi(start_date):
    # alpha_vantage getting cpi data
    params = {
        'function': 'CPI',
        'interval': 'monthly',
        'apikey': av_api_key
        }

    response = requests.get(alpha_vantage_url, params)
    data = response.json()
    cpi = pd.DataFrame(data['data'])
    cpi = cpi.iloc[::-1]
    cpi.set_index('date', inplace=True)
    cpi = cpi[start_date:]
    cpi.reset_index(inplace=True)
    starting_cpi = cpi['value'].loc[0]

    return cpi, starting_cpi


In [ ]:
def get_ffr(start_date):
    # alpha_vantage getting federal funds rate
    params = {
        'function': 'FEDERAL_FUNDS_RATE',
        'interval': 'daily',
        'apikey': av_api_key
        }

    response = requests.get(alpha_vantage_url, params)
    data = response.json()
    ffr = pd.DataFrame(data['data'])
    ffr = ffr.iloc[::-1]
    ffr.set_index('date', inplace=True)
    ffr = ffr[start_date:]
    ffr.reset_index(inplace=True)

    return ffr

In [ ]:
def get_earnings(ticker):
    # We are going to alpha_vantage for this
    params = {
        'function': 'EARNINGS',
        'symbol': ticker,
        'outputsize': 'full',
        'datatype': 'json',
        'apikey': av_api_key
        }

    try:
        response = requests.get(alpha_vantage_url, params)
        data = response.json()
        quarterly_earnings = pd.DataFrame(data['quarterlyEarnings'])
        quarterly_earnings['fiscalDateEnding'] = pd.to_datetime(quarterly_earnings['fiscalDateEnding'])

        return quarterly_earnings

    except Exception as e:
        print("An exception occurred: ", e)

        return e

In [ ]:
def create_frame(stock_ticker, timeframe, future_offset, index_length, merged_economic_data, period='max'):
    #creating and forming the master frame, converting data to float (except for ticker which will be done later)
    #data taken from yahoo finance
    ### TAKING OUT DATE COLUMNS AS THEY COULD CORRUPT RESULTS SINCE DATE FOR PREDICTION DATA WILL ALWAYS BE CURRENT

    # Create the initial dataframe
    try:
        ticker = yf.Ticker(stock_ticker)

        #initializing the frame
        df = pd.DataFrame(ticker.history(period=period))
    except ValueError as e:
        if str(e) == f"{stock_ticker}: No data found, symbol may be delisted":
            print(f"Symbol {symbol} is not available.")
        else:
          raise e

    if df.empty:
        raise DataError(f"No data available for {ticker}.")
    elif len(df) < index_length:
        raise DataError(f"Insufficient data available for {ticker}.")
    elif df.isna().values.any():
        raise DataError(f"Null values found for {ticker}.")

    # Now create the earnings dataframe and prepare it
    try:
        df_earnings = get_earnings(stock_ticker)
        df_earnings = df_earnings.iloc[::-1]
        df_earnings.reset_index(drop=True, inplace=True)
    except Exception:
        pass

    df_earnings['reportedEPS'] = df_earnings['reportedEPS'].replace('None', 0.0)
    df_earnings['estimatedEPS'] = df_earnings['estimatedEPS'].replace('None', 0.0)
    df_earnings['surprise'] = df_earnings['surprise'].replace('None', 0.0)
    df_earnings['surprisePercentage'] = df_earnings['surprisePercentage'].replace('None', 0.0)

    # Now combine earnings and also cpi and federal funds rate data

    #initialize earnings to 0.0
    df['reportedEPS'] = 0.0
    df['estimatedEPS'] = 0.0
    df['surprise'] = 0.0
    df['surprisePercentage'] = 0.0

    #set up a date column for our loops
    df['date'] = pd.to_datetime(df.index.date)
    df['date'] = df['date'].dt.strftime('%Y-%m-%d')

    df.reset_index(drop=True, inplace=True)

    #earnings loop
    for i in range(len(df)):
        target_date = df.loc[i, 'date']

        #add in earnings
        for j in range(len(df_earnings)):
            reported_date = df_earnings.loc[j, 'reportedDate']
            if reported_date == target_date:
                df.loc[i, 'reportedEPS'] = df_earnings.loc[j, 'reportedEPS']
                df.loc[i, 'estimatedEPS'] = df_earnings.loc[j, 'estimatedEPS']
                df.loc[i, 'surprise'] = df_earnings.loc[j, 'surprise']
                df.loc[i, 'surprisePercentage'] = df_earnings.loc[j, 'surprisePercentage']
            else:
                continue

    #merge in economics
    df = pd.merge(df, merged_economic_data, on='date', how='left')
    df.fillna(method='ffill', inplace=True)

    df = df.drop('date', axis=1)
    df.reset_index(drop=True, inplace=True)

    #converting some things to type float and calculating "future close" which is a column for price "future_offset" days in the future
    df['Dividends'] = df['Dividends'].astype('float')
    df['Stock Splits'] = df['Stock Splits'].astype('float')
    df['Volume'] = df['Volume'].astype('float')
    df['Future Close'] = df['Close'].shift(-(future_offset))

    #getting most recent close
    most_recent_close = df['Close'].iloc[-1]

    del df_earnings

    return df, most_recent_close

In [ ]:
def create_sets(frame, timeframe, future_offset, input_scaler, number_of_selections=20):
    #Creating the X and ys for each individual stock
    # THIS FUNCTION RETURNS A LIST CONTAINING FRAMES FOR ONE STOCK

    #initializing train, test, and validation
    train_X = []
    train_y = []
    validation_X = []
    validation_y = []
    prediction_set = []

    #creating the prediction set and adding one_hot
    predict_set = frame.iloc[-timeframe:].drop(['Future Close'], axis=1)
    predict_set.reset_index(drop=True, inplace=True)
    predict_set = input_scaler.transform(predict_set)
    prediction_set.append(predict_set)

    #dropping future_offset as it contains unpredictable values for future_close and move_level
    frame = frame.iloc[:-future_offset]

    #creating a sample size to be divided among sets
    #number_of_selections = int((len(frame)) / ((timeframe) / 3))

    training_size = int(number_of_selections * .8)
    training_selections = random.sample(range(len(frame)-timeframe), training_size)
    for i in training_selections:
        X_frame = frame.iloc[i:i+timeframe].drop(['Future Close'], axis=1)
        X_frame.reset_index(drop=True, inplace=True)
        X_frame = input_scaler.transform(X_frame)
        y = frame.iloc[i+timeframe]['Future Close']
        train_X.append(X_frame)
        train_y.append(y.item())

    validation_size = int(number_of_selections * .2)
    validation_selections = random.sample(range(len(frame)-timeframe), validation_size)
    for i in validation_selections:
        X_frame = frame.iloc[i:i+timeframe].drop(['Future Close'], axis=1)
        X_frame.reset_index(drop=True, inplace=True)
        X_frame = input_scaler.transform(X_frame)
        y = frame.iloc[i+timeframe]['Future Close']
        validation_X.append(X_frame)
        validation_y.append(y.item())

    return train_X, train_y, validation_X, validation_y, prediction_set

In [ ]:
def prepare_data(combined_frames, timeframe, future_offset, input_scaler, number_of_selections=20): #removed scaler
    # The catch all function to run the functions that organize the data
    # will return list of dataframes that are scaled
    train_data = []
    train_labels = []
    validation_data = []
    validation_labels = []
    predict_data = []
    for each in combined_frames:
        #scaled_frame = scale_data(combined_frames[each])#, scaler)
        t_X, t_y, v_X, v_y, p_s = create_sets(combined_frames[each], timeframe, future_offset, input_scaler, number_of_selections)
        train_data.extend(t_X)
        train_labels.extend(t_y)
        validation_data.extend(v_X)
        validation_labels.extend(v_y)
        predict_data.extend(p_s)

    return train_data, train_labels, validation_data, validation_labels, predict_data

In [ ]:
def combine_with_indexes(index_dict, frame_dict):
        #redoing the dictionary of frames to add stock indexes for the same date ranges
    new_frame_dict = {
        ticker: pd.concat([frame, pd.concat(index_dict.values(), axis=1)], axis=1)
        for ticker, frame in frame_dict.items()
    }

    return new_frame_dict

In [ ]:
class DataError(Exception):
    #creating an exception class to ensure adequate data per stock in list
    pass

In [ ]:
def array_stacker(dataframe_list):
    #converting input dataframes into appropriate tensor form
    return tf.stack([dataframe_list])

def label_stacker(label_list):
    #converting labels into appropriate tensor form
    return tf.stack([label_list[each] for each in range(len(label_list))])

In [ ]:
def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

In [ ]:
indices = {
    'SPX':'^GSPC',
    'Dow':'^DJI',
    'Nasdaq':'^IXIC',
    'Russell':'^RUT',
    'NYSE':'^NYA',
    'AMEX':'^XAX',
    'XLF':'XLF',
    'XLE':'XLE',
    'XLU':'XLU',
    'XLB':'XLB',
    'XLI':'XLI',
    'XLY':'XLY',
    'XLV':'XLV',
    'SMH':'SMH',
    'XLP':'XLP',
    'XTL':'XTL',
    'IYR':'IYR',
    'VIX':'^VIX'
    }

In [ ]:
timeframe = 20
period = '5y'
future_offset = 1
number_of_selections = 100

# Create dictionary of dataframes for indices
index_dict = {}
start_date = str(0)
for key, value in indices.items():
    df = get_indices(value, period)
    if key == 'SPX':
        start_date = df.index[0].strftime('%Y-%m-%d')
    df.reset_index(drop=True, inplace=True)
    index_dict[key] = df

index_length = len(index_dict['SPX'])-1

In [ ]:
# let's define a few global variable dataframes to reuse
ffr = get_ffr(start_date)
cpi, starting_cpi = get_cpi(start_date)
cpi.loc[cpi['value'].isna(), 'value'] = starting_cpi

merged_economic_data = pd.merge(ffr, cpi, on='date', how='left')
merged_economic_data['value_y'].fillna(method='ffill', inplace=True)
merged_economic_data.loc[merged_economic_data['value_y'].isna(), 'value_y'] = starting_cpi
merged_economic_data.rename(columns={'value_x': 'ffr', 'value_y': 'cpi'}, inplace=True)

x = 0
frame_dict = {}
close_dict = {}

sector_columns = [stocklist.columns[1:].tolist()]

for idx, row in stocklist.iterrows():
    time.sleep(15)
    ticker = row['Ticker']
    if x < 100:
        print(ticker, end=", ")
        x += 1
    else:
        print(ticker)
        print()
        x = 0
        time.sleep(600)

    try:
        frame, most_recent_close = create_frame(ticker, timeframe, future_offset, index_length, merged_economic_data, period)
        for each in sector_columns:
            frame[each] = row[each]
        frame_dict[ticker] = frame
        close_dict[ticker] = most_recent_close

    except Exception as e:
        print(f"Error for {ticker}: {e}")
        pass

del stocklist

In [ ]:
combined_frames = combine_with_indexes(index_dict, frame_dict)

del index_dict

scaler = MinMaxScaler()
scale_input_frame = pd.concat(combined_frames)
scale_input_frame = scale_input_frame.drop(columns=['Future Close'])
input_scaler = scaler.fit(scale_input_frame)

train_data, train_labels, validation_data, validation_labels, predict_data = prepare_data(combined_frames, timeframe, future_offset, input_scaler, number_of_selections)
del combined_frames

In [ ]:
scale_labels = train_labels + validation_labels
output_scaler = scaler.fit(np.array(scale_labels).reshape(len(scale_labels), 1))

In [ ]:
input_X = array_stacker(train_data)
input_y = output_scaler.transform(np.array(train_labels).reshape(-1, 1))

validation_X = array_stacker(validation_data)
validation_y = output_scaler.transform(np.array(validation_labels).reshape(-1, 1))

predict_X = array_stacker(predict_data)

In [ ]:
input_X = tf.reshape(input_X, (input_X.shape[1], input_X.shape[2], input_X.shape[3]))
input_y = tf.reshape(input_y, (input_y.shape[0], 1))
validation_X = tf.reshape(validation_X, (validation_X.shape[1], validation_X.shape[2], validation_X.shape[3]))
validation_y = tf.reshape(validation_y, (validation_y.shape[0], 1))
predict_X = tf.reshape(predict_X, (predict_X.shape[1], predict_X.shape[2], predict_X.shape[3]))

In [ ]:
model_3 = Sequential([
        tf.keras.layers.LSTM(32, dropout=.25, activation='tanh', return_sequences=True),# return_sequences=True)),
        tf.keras.layers.LSTM(16, dropout=.25, activation='tanh', return_sequences=True),
        tf.keras.layers.LSTM(8),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(1, activation='linear')
])

model_3.compile(optimizer='Adam', loss='mse', metrics=[rmse])

In [ ]:
batch_size = 32
epochs = 10000

history_3 = model_3.fit(x=input_X,
              y=input_y,
              batch_size=batch_size,
              epochs=epochs,
              verbose=2,
              validation_data=(validation_X, validation_y),
              callbacks=EarlyStopping(monitor='val_rmse', restore_best_weights=True, patience=150)
            )

In [ ]:
plt.plot(history_3.history['rmse'])
plt.plot(history_3.history['val_rmse'])
plt.title('Model RMSE')
plt.xlabel('Epoch')
plt.ylabel('RMSE')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()


# Extract the RMSE values from the history
rmse_values3 = history_3.history['val_rmse']

# Extract the final and lowest RMSE values
print("Last: ", rmse_values3[-1], "Best: ", min(rmse_values3))

In [ ]:
predictions = output_scaler.inverse_transform(model_3.predict(predict_X))

In [ ]:
results = pd.DataFrame()
results['Ticker'] = frame_dict.keys()
results['Most Recent Close'] = [close_dict[each] for each in results['Ticker']]
results['Predicted Price'] = predictions
results['Percent Change'] = ((results['Predicted Price'] - results['Most Recent Close']) / results['Most Recent Close']) * 100

In [ ]:
print(results.sort_values(by='Percent Change', ascending=False))

In [ ]:
shortened_results = results.drop(results[results['Most Recent Close'] < 1.00].index)
print(shortened_results.sort_values(by='Percent Change', ascending=False))

In [ ]:
# Code for saving shortened_results into Excel
writer = pd.ExcelWriter('drive/My Drive/AI Predictions.xlsx', engine='openpyxl')
tz = pytz.timezone('US/Eastern')
today = datetime.now(tz).strftime('%Y-%m-%d')
shortened_results.to_excel(writer, sheet_name=today, index=False)

# Accessing the workbook and active sheet
workbook  = writer.book
worksheet = writer.sheets[today]

# Setting column width based on the maximum length of each column header
for i, column in enumerate(shortened_results.columns):
    column_length = max(shortened_results[column].astype(str).map(len).max(), len(column))
    worksheet.column_dimensions[openpyxl.utils.get_column_letter(i+1)].width = column_length + 2

writer.close()